## 手机基站数据预处理

In [13]:
FILES_DIR = {
    # "base_station":"E:/MSE/VAG/VIS2021/温州数据/手机基站数据/trajs集合/basestation.json",
    # "phone_dir":"E:/MSE/VAG/VIS2021/温州数据/手机基站数据/TOP10000Trajs/",
#     # 基站
    "base_station" : "C:/Users/vag/Desktop/ykj/手机基站数据/basestation.json",
#     # 手机记录文件夹
    "phone_dir": "./DP/phoneData/trajs/",
    "save_phone_dir":"../data_in_use/",
    "slice_phone_dir":"../originPhoneData/"
#     "base_station" : "/Volumes/Age's SSD 1/SSD/数据/手机基站/basestation.json",
#     # 手机记录文件夹
#     "phone_dir": "/Volumes/Age's SSD 1/SSD/数据/手机基站/phonetraj14-15/phonetraj0114/"
    # "phone": "/Volumes/Age's SSD 1/SSD/数据/手机基站/phonetraj14-15/phonetraj0114/mpt2014114 1140010sortbyid.txt"
}
import pandas as pd
import json

### 基站数据 ↓

手机基站

每条文档包括基站的坐标、基站的 LAC 码和 CELLID，通过 LAC 码和 CELLID 可以唯一定位一座手机基站。

In [2]:
df_station = pd.read_json(FILES_DIR["base_station"])
import util
config = util.getJsonData("./config.json")
b = config['area']
print(len(df_station))
df_station_filter_in_loca = df_station
df_station_filter_in_loca = df_station_filter_in_loca[ df_station_filter_in_loca['longitude'] > b['minLng'] ]
df_station_filter_in_loca = df_station_filter_in_loca[ df_station_filter_in_loca['longitude'] < b['maxLng'] ]
df_station_filter_in_loca = df_station_filter_in_loca[ df_station_filter_in_loca['latitude'] > b['minLat'] ]
df_station_filter_in_loca = df_station_filter_in_loca[ df_station_filter_in_loca['latitude'] < b['maxLat'] ]
df_station = df_station_filter_in_loca
print(len(df_station))

102004
6051


### 手机轨迹 ↓

每条记录只包含手机的 IMEI 码、日志的时间戳、记录这条数据的手机基站的 LAC 码和 CELLID

轨迹文件按照 10min时间块存储,

202091150448482;2014-01-13 23:52:58.37,22492 24185;2014-01-13 23:53:00.25,22492 24185;

In [3]:
# 读取手机基站轨迹原始文件
def read_phone_raw_file(filename):
    phone_trajs = []
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            phone_trajs.append(
                split_line_to_trajectory(line)
            )
    return phone_trajs

def split_line_to_trajectory(line):
    arr = line.split(";")
    trajectory = {
        "IMEI": arr[0],
        "POINT": []
    }
    for point in arr[1:-1]:
        parr = point.split(",")
        trajectory["POINT"].append({
            "time": parr[0],
            "sid": parr[1]
        })
    return trajectory
# sample_line = "202091150448482;2014-01-13 23:52:58.37,22492 24185;2014-01-13 23:53:00.25,22492 24185;"
# print(split_line_to_trajectory(sample_line))

In [4]:
from os import walk

# 获取文件列表
def list_files():
    phone_raw_files = []
    for (dirpath, dirnames, filenames) in walk(FILES_DIR["phone_dir"]):
        phone_raw_files.extend(filenames)
    return phone_raw_files

# 读取所有文件中的数据
def read_files():
    files = list_files()
    BASE_PATH = FILES_DIR["phone_dir"]
    print("files: " + str(len(files)))
    print(files[0])
    f = BASE_PATH + files[0]
    return read_phone_raw_file(f)

### 数据格式

In [5]:
import functools
def compare(x, y):
    if x["time"] > y["time"]:
        return 1
    if x["time"] < y["time"]:
        return -1
    return 0
# 连续两个相同基站时删除重复基站记录,保留最后一个元素来确保知道最后的时间点
def filterRepeatedPoints(points):
    last = ''
    for i in range(len(points)-2,-1,-1):
        p = points[i]
        if last == p['sid']:
            points.pop(i)
        else:
            last = p['sid']

In [9]:
#  策略
#  1. 如果找不到对应基站，该点 筛去
def add_lnglat_to_phone_trajectory(phone):
    points = phone['POINT']
    new_points = []
    for p in points:
        try:
            lac_cell = p['sid'].split(' ')
            lac = int(lac_cell[0])
            cell = int(lac_cell[1])
            df_find = find_in_station_df(lac, cell)
        except:
            continue
        else:
            p['longitude'] = df_find[0]
            p['latitude'] = df_find[1]
            del p['sid']
    phone['points'] = phone.pop('POINT')
    
    for p in phone['points']:
        if('sid' not in p):
            new_points.append(p)
    phone['points'] = new_points
            


def find_in_station_df(lac, cell):
    df = df_station
    df_find = df.loc[(df['lac'] == lac) & (df['cell'] == cell)]
    # print(df_find['longitude'])
    if(len(df_find) != 1):
        raise Exception("Cant Find Station!")
    return df_find[['longitude','latitude']].values[0]

def write_json_to_file(json_obj, filename='../data_in_use/phone.json'):
    # path = ''
    # filename = path + filename
    with open(filename, 'w') as outfile:
        json.dump(json_obj, outfile)
    

In [7]:

def pack_data(filename,num = 10000):
    c = 0
    new_phones = []
    for phone in phones[1:num]:
        #先给points按照时间排序===========================================
        phone["POINT"] = sorted(phone["POINT"], key=functools.cmp_to_key(compare))
        filterRepeatedPoints(phone["POINT"])
#         ==========================================================
        add_lnglat_to_phone_trajectory(phone)
        c = c + 1
        if(len(phone['points']) > 0):
            new_phones.append(phone)
        if(c % 1000 == 0):
            print('doing ' + str(c) + ' ...')

    write_json_to_file(new_phones,filename)
    print('done!')
## 手机用户轨迹
files = list_files()
# files.sort(reverse=True)
print("files: " + str(len(files)))
slicePhonePath = FILES_DIR["slice_phone_dir"]
for file in files:
    f = BASE_PATH + file
    print("当前执行文件",f)
    phones = read_phone_raw_file(f)
    pack_data(slicePhonePath+file, len(phones))
print("手机基站数据处理完成======   `")


 ...
doing 4000 ...
doing 5000 ...
doing 6000 ...
doing 7000 ...
doing 8000 ...
doing 9000 ...
done!
当前执行文件 ./DP/phoneData/trajs/TOP10000mpt2014114 1140810sortbyid.txt
doing 1000 ...
doing 2000 ...
doing 3000 ...
doing 4000 ...
doing 5000 ...
doing 6000 ...
doing 7000 ...
doing 8000 ...
doing 9000 ...
done!
当前执行文件 ./DP/phoneData/trajs/TOP10000mpt2014114 1140820sortbyid.txt
doing 1000 ...
doing 2000 ...
doing 3000 ...
doing 4000 ...
doing 5000 ...
doing 6000 ...
doing 7000 ...
doing 8000 ...
doing 9000 ...
done!
当前执行文件 ./DP/phoneData/trajs/TOP10000mpt2014114 1140830sortbyid.txt
doing 1000 ...
doing 2000 ...
doing 3000 ...
doing 4000 ...
doing 5000 ...
doing 6000 ...
doing 7000 ...
doing 8000 ...
doing 9000 ...
done!
当前执行文件 ./DP/phoneData/trajs/TOP10000mpt2014114 1140840sortbyid.txt
doing 1000 ...
doing 2000 ...
doing 3000 ...
doing 4000 ...
doing 5000 ...
doing 6000 ...
doing 7000 ...
doing 8000 ...
doing 9000 ...
done!
当前执行文件 ./DP/phoneData/trajs/TOP10000mpt2014114 1140850sortbyid.txt


In [14]:
# 将轨迹数据集成到一个文件里面
import util

BASE_PATH = FILES_DIR["save_phone_dir"]
slicePhonePath = FILES_DIR["slice_phone_dir"]
slicePhoneFiles = []
for (dirpath, dirnames, filenames) in walk(slicePhonePath):
    slicePhoneFiles.extend(filenames)
phones = {}
for f in slicePhoneFiles:
    data = util.getJsonData(slicePhonePath+f)
    for traj in data:
        if traj["IMEI"] in phones:
            phones[traj["IMEI"]]["points"].extend(traj["points"])
        else:
            phones[traj["IMEI"]] = traj
write_json_to_file(phones,BASE_PATH+"phone.json")


VAUD Case 部分

### 1. 获取某用户一天中所有轨迹点数据

In [8]:
## Case： 单独获取某个用户的轨迹数据
# 整体跑完约 15 min
def get_one_traj_with_id(phoneId):
  files = list_files()
  BASE_PATH = FILES_DIR["phone_dir"]
  oneTraj = []
  for filename in files:
    if(filename == '.DS_Store'):
      continue
    file = BASE_PATH + filename
    print(file)
    with open(file) as f:
      lines = f.readlines()
      for line in lines:
          TrajID = line.split(";")[0]
          # print("TrajID",TrajID)
          if (TrajID == phoneId):
            print(line)
            print('-'*50)
            oneTraj.append(line)
            break
    f.close()
  print('-'*100)
  print(oneTraj)
  with open('../data_in_use/phone_' + phoneId + '.txt', 'w') as f:
    for line in oneTraj:
      f.writelines(line)
    f.close()

def pack_one_traj(phoneId):
    with open('../data_in_use/phone_' + phoneId + '.txt', 'r' ) as f:
        segments = f.readlines()
    trajectory = {
      "id": phoneId,
      "segments": []
    }
    for line in segments:
      segment = split_line_to_trajectory(line)
      add_lnglat_to_phone_trajectory(segment)
      if(len(segment['points']) > 0):
        trajectory['segments'].append(segment['points'])
    with open('../node-server-data/phone_' + phoneId + '.json', 'w' ) as f:
      json.dump(trajectory, f)
    print('Done!')

# get_one_traj_with_id('460022584127733')
# pack_one_traj('460022584127733')